[View in Colaboratory](https://colab.research.google.com/github/fatmas1982/fh2017/blob/master/Copy_of_fetch_20newsgroups3.ipynb)

In [0]:
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
from collections import Counter
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot') 
import numpy as np
import scipy.stats.stats as st
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.wsd import lesk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
#from stemming.porter2 import stem
from nltk import PorterStemmer
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from string import digits
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import csv

import re
from nltk import word_tokenize
import string
!pip install gensim
import  gensim.models as md
from gensim.models.phrases import Phrases, Phraser


In [0]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting up python-apt (1.4.0~beta3build2) ...
Setting up python3-dbus (1.2.4-1build

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
#https://colab.research.google.com/notebook#fileId=1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q&scrollTo=c99EvWo1s9-x
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!ls 


datalab   file.txt   nytimes_news_articles.txt	stopwords.txt
drive_PH  nltk_data  scikit_learn_data


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving nytimes_news_articles.txt to nytimes_news_articles.txt


In [0]:
!mkdir -p drive_PH
!google-drive-ocamlfuse drive_PH

In [0]:
path_database='./drive_PH/Colab Notebooks/' 

pragraph_index='pragraph_index_20.csv'
Sentences='Sentences_20.csv'
Sentences_not_stops='Sentences_not_stops_20.csv'
lesk='lesk_20.csv'

In [0]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [0]:
documents[0]
dataset.filenames

array(['/content/scikit_learn_data/20news_home/20news-bydate-train/talk.politics.mideast/76141',
       '/content/scikit_learn_data/20news_home/20news-bydate-train/alt.atheism/53281',
       '/content/scikit_learn_data/20news_home/20news-bydate-train/talk.politics.mideast/76350',
       ...,
       '/content/scikit_learn_data/20news_home/20news-bydate-train/rec.sport.baseball/105105',
       '/content/scikit_learn_data/20news_home/20news-bydate-train/comp.sys.mac.hardware/51575',
       '/content/scikit_learn_data/20news_home/20news-bydate-train/rec.sport.baseball/104908'],
      dtype='<U89')

In [0]:
'''
Write Excell sheet
'''
def save_file_to_database(data_rows,path_database,file_databbase,header_list):#header_list=['index','text']
    outfile = open(path_database+file_databbase,'w')
    writer=csv.writer(outfile)
    #header_list=['uuid','paragraph','doc_id']
    i=0
    for line in data_rows:
        row=[i,line]#,'paragraph no.'+str(i)]
        if i==0:
            
            writer.writerow(header_list)
            writer.writerow(row)
        else:
            #print('ff')
            writer.writerow(row)
        i+= 1
        #outfile.close()

In [0]:
save_file_to_database(documents,path_database,pragraph_index,header_list)

In [0]:
def read_cvs_by_pands(path_database,file_databbase,index_col, header):
    return pd.read_csv(path_database+file_databbase,index_col=index_col,header=header)

In [0]:
def pragraph_to_setnences(str):
    return sent_tokenize(str)

In [0]:
def txt_pragraphs(str):
    pragraphs = str.split("\n\n")
    return pragraphs

In [0]:
def read_text_from_database(path_database,file_databbase):
    queue_paragraph=[]
    #f = open(sys.argv[1], 'rt')
    outfile = open(path_database+file_databbase,'rt')
    try:
                
        reader=csv.reader(outfile)
        for row in reader:
            queue_paragraph.append(row)
            #print (row)
    finally:
        print ("row")
        outfile.close()
        
    return queue_paragraph

In [0]:
def stopwords_list():
    stopwordsFile = open('./stopwords.txt')
    stopwordsFile.seek(0)
    stopwordsV1 = stopwordsFile.readlines()
    stopwordsV2 = []
    for sent in stopwordsV1:
        sent.replace('\n', '')
        new_word = sent[0:len(sent) - 1]
        stopwordsV2.append(new_word.lower())
    return stopwordsV2

In [0]:
new_stop_words = ['the', 'that', 'to', 'as', 'there', 'has', 'and', 'or', 'is', 'not', 'a', 'of', 'but', 'in', 'by', 'on', 'are', 'it', 'if','what','where','how','when']
new_stop_words2=['--','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now','even','until','then','must']
numbers=[1,2,3,4,5,6,7,8,9]
stopwordsV2=stopwords_list()
#stemmer = SnowballStemmer("english", ignore_stopwords=True)
def remove_stopword_sentences(str):
   
            
    list_word=[]
    tokenizer = RegexpTokenizer("[\w']+")
    
    words=tokenizer.tokenize(str)
    for word in words:
        new_word = word.encode('ascii', 'ignore').decode('utf-8')
        if new_word != '':
    
            english_stops = set(stopwords.words('english'))
           
            list_word=[new_word for new_word in words if new_word.lower() not in english_stops
                       and new_word.lower() not in new_stop_words 
                       and new_word.lower() not in new_stop_words2 
                       and  not new_word.lower().isdigit() 
                       and new_word.lower() not in digits 
                       and new_word.lower() not in  numbers and word.lower() not in stopwordsV2
                       and new_word.lower() not in string.punctuation]
    
  
    
    return list_word#(stem(setem_word for setem_word in  ([word for word in words if word not in english_stops and word not in new_stop_words])))

In [0]:
!ls nytimes_news_articles.txt

nytimes_news_articles.txt


In [0]:
import pandas as pd
def courps_to_CSV_docs():
    #Reading the news articles file
    nyTimesFile = open('./nytimes_news_articles.txt', encoding='latin-1')
    nyTimesFile.seek(0)
    nyTimesV1 = nyTimesFile.readlines()
    nyTimesTemp = []
    nyTimesURL = []

    for i in range(0, len(nyTimesV1)-1):
        if re.findall('URL', nyTimesV1[i]) == []:
            sent = sent + nyTimesV1[i]
            if (re.findall('URL', nyTimesV1[i+1]) != []) and (i+1 < len(nyTimesV1)):
                nyTimesTemp.append(sent.strip())
        else:
            sent = ''
            nyTimesURL.append(nyTimesV1[i])

    for i in range(0, len(nyTimesTemp)):
        nyTimesTemp[i] = nyTimesTemp[i]+'articleID'+str(i)
    print(len(nyTimesTemp))
    header_list=['index','text']
    save_file_to_database(nyTimesTemp,path_database,pragraph_index,header_list)
    '''for i in range(1):
        print(i,"============================================")
        print("============================================")'''
    #nytimes = preProcessor(nyTimesTemp)
    print("============================================")
    #print(nytimes)

In [0]:
#courps_to_CSV_docs()

61


In [0]:
header_list=['index','text']
#save_file_to_database(documents,'/',pragraph_index,header_list)

In [0]:
paragraphs=read_cvs_by_pands(path_database,pragraph_index,None,0)
paragraphs

,index,text
0,0,Well i'm not sure about the story nad it did s...
1,1,"\n\n\n\n\n\n\nYeah, do you expect people to re..."
2,2,Although I realize that principle is not one o...
3,3,Notwithstanding all the legitimate fuss about ...
4,4,"Well, I will have to change the scoring on my ..."
5,5,"\n \nI read somewhere, I think in Morton Smit..."
6,6,\nOk. I have a record that shows a IIsi with ...
7,7,\n\n\nSounds like wishful guessing.\n\n\n\n\n'...
8,8,Nobody is saying that you shouldn't be allowe...
9,9,\n I was wondering if anyone can shed any lig...


In [0]:
!ls

datalab   file.txt   nytimes_news_articles.txt	stopwords.txt
drive_PH  nltk_data  scikit_learn_data


In [0]:
import nltk
nltk.download('punkt')
def paragraphs_to_sentece(pragraphs):
    sentenses_list=[]
    
    for index_p in  range(len(pragraphs)):
        #print(index_p)
        #print("pppppppppppppppppppp")
        #print(pragraphs[index_p])
        if pragraphs[index_p] is not None:
          #print("setnences")
          #p1=txt_pragraphs(pragraphs[index_p])
          setnences=pragraph_to_setnences(str(pragraphs[index_p]))
          #print("sssssssssssssssssssssssssss")
          #print("setnences")#,setnences)

          for indexs in range(len(setnences)):
              row=[]
              #print(setnences)
              row.append(index_p)
              row.append(indexs)
              row.append(setnences[indexs])
              sentenses_list.append(row)
    header_list=['index_P','index_sent','sentence']
    df = pd.DataFrame(sentenses_list, columns=header_list)#, index=index)
    #df

    #print(sentenses_list)
    df.to_csv(path_database+Sentences, encoding='utf-8', index=False)
    #save_file_to_database(sentenses_list,path_database,"Sentences.csv",header_list)        
    return df

paragraphs_to_sentece(paragraphs.text)

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [99]:
setences=read_cvs_by_pands(path_database,Sentences,None,0)
setences

,index_P,index_sent,sentence
0,0,0.0,Well i'm not sure about the story nad it did s...
1,0,1.0,What\nI disagree with is your statement that t...
2,0,2.0,That is rediculous.
3,0,3.0,The U.S. media is\nthe most pro-israeli media ...
4,0,4.0,Having lived in Europe\nI realize that inciden...
5,0,5.0,The U.S. media as a whole seem to try to\nigno...
6,0,6.0,The U.S. is subsidizing Israels existance and ...
7,0,7.0,So I think\nthat might be a reason they report...
8,0,8.0,"What is a shame is that in Austria, daily repo..."
9,0,9.0,"After all, look how the Jews are treating othe..."


In [0]:
nltk.download('stopwords')
def  sentece_Not_stop_word(setences):
    #words_list=[]
    sentenses_list=[]
    
    for index_s in  range(len(setences)):
            
          #print("Sentence No. ",index_s,": ",setences.loc[index_s]['sentence'],"\n")
          words=remove_stopword_sentences(str(setences.loc[index_s]['sentence']))
          wordsent=''
          row=[]
          

          row.append(setences.loc[index_s]['index_P'])
          row.append(setences.loc[index_s]['index_sent'])
          row.append(words)
          sentenses_list.append(row)
    header_list=['index_P','index_sent','words_not_stop']
    df = pd.DataFrame(sentenses_list, columns=header_list)#, index=index)
    #df

    #print(sentenses_list)
    df.to_csv(path_database+Sentences_not_stops, encoding='utf-8', index=False)
    #save_file_to_database(sentenses_list,path_database,"Sentences.csv",header_list)        
    return df

sentece_Not_stop_word(setences)

In [0]:
'''
this function for compute lesk of word in sentence
'''

def lesk_word_sentence(sentence,word):
    from nltk.wsd import lesk
    lesk_synset=''
    #lesks= []
    #for word in words:
    #disambiguated=lesk(context_sentence=sentence, ambiguous_word=word)
    disambiguated=lesk(sentence,word, 'n')
    #print(disambiguated)
    #if disambiguated is not None:
    lesk_synset=disambiguated
    #else:
    #lesk_synset=0
    #print("Word is: ",word,"\n LESK: ",lesk(sentence,word, 'n'),"\n Sentence: ",sentence )
        
    return lesk_synset

#lesk("Computer science is a discipline that spans theory and practice","science")

#sent = 'people should be able to marry a person of their choice'.split()
#lesk(sent, 'able')

In [118]:
df_Sentences_not_stops=read_cvs_by_pands(path_database,Sentences_not_stops,None,0)
df_Sentences_not_stops

,index_P,index_sent,words_not_stop
0,0,0.0,"['Well', ""i'm"", 'sure', 'story', 'nad', 'seem'..."
1,0,1.0,"['disagree', 'statement', 'U', 'Media', 'ruin'..."
2,0,2.0,['rediculous']
3,0,3.0,"['U', 'media', 'pro', 'israeli', 'media', 'wor..."
4,0,4.0,"['lived', 'Europe', 'realize', 'incidences', '..."
5,0,5.0,[]
6,0,6.0,"['U', 'subsidizing', 'Israels', 'existance', '..."
7,0,7.0,"['think', 'might', 'reason', 'report', 'clearl..."
8,0,8.0,[]
9,0,9.0,"['look', 'Jews', 'treating', 'races', 'got', '..."


In [0]:
nltk.download('wordnet')
def sent_lesks():
    df_Sentences=read_cvs_by_pands(path_database,Sentences,None,0)
    df_Sentences_not_stops=read_cvs_by_pands(path_database,Sentences_not_stops,None,0)
    sentences=df_Sentences.sentence
    words_not_stop=df_Sentences_not_stops.words_not_stop#[0]#['words_not_stop'][0]
    lesk_df_list=[]
    file=open('./file.txt', 'a+')
    #print(len(df_Sentences))
    #print(len(df_Sentences_not_stops))
    for i in range(len(df_Sentences)):
        lesks=[]
        lesks_name=[]
        row=[]
        if words_not_stop[i]!='[]':

            words_not_stop[i]=words_not_stop[i].replace("'",'').replace("[",'').replace("]",'')
            sentences[i]=sentences[i].replace("'",'').replace("[",'').replace("]",'')       
            words_not_stop_list=words_not_stop[i].split(',')
            #print(words_not_stop_list)
            #print(sentences[i])
            for word in words_not_stop_list:
                #print(word)
                l=lesk_word_sentence(sentences[i],word)
                #print("l",l)
                if l is not None:
                    lesks.append(l)
                    lesks_name.append(l.name())
        #print("888888888888888888888888888888888888888888")
        row.append(df_Sentences_not_stops.index_P[i])
        row.append(df_Sentences_not_stops.index_sent[i])
        row.append(lesks)
        row.append(lesks_name)
        lesk_df_list.append(row)
        #print(i)
        file.write(str(i))

    #print(lesk_df_list)
    header_list=['index_P','index_sent','lesks','lesks_name']
    df = pd.DataFrame(lesk_df_list, columns=header_list)
    
    df.to_csv(path_database+"lesks.csv", encoding='utf-8', index=False)  
    #return df




sent_lesks()

[nltk_data] Downloading package wordnet to /content/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
l=read_cvs_by_pands(path_database,'lesks.csv',None,0)
l

,index_P,index_sent,lesks,lesks_name
0,0,0,[Synset('washington.n.05')],['washington.n.05']
1,0,1,[],[]
2,0,2,[],[]
3,0,3,[],[]
4,0,4,[],[]
5,0,5,[Synset('pitching.n.01')],['pitching.n.01']
6,0,6,[Synset('large.n.01')],['large.n.01']
7,0,7,[],[]
8,0,8,[],[]
9,0,9,[],[]


In [0]:
!ls 

datalab   file.txt   nytimes_news_articles.txt	stopwords.txt
drive_PH  nltk_data  scikit_learn_data
